In [ ]:
%matplotlib inline
!git pull
import os;os.environ['LAMBDA_PARQUET_TICK_DB']=rf'X:\lambda_data'
session_name='Phd-Script'
import time
import datetime
import tqdm

In [ ]:
start=time.time()
from notebooks.lambda_imports import *
dill.load_session(session_name)

TRAIN_DQN=False
DEBUG=False
PLOT_CANDLES=True
HURST_TEST=True
PARAMETER_TUNING_BENCHMARK_INITIAL=False
PARAMETER_TUNING_BENCHMARK=True

elapsed = time.time()-start

In [ ]:
seed_number=666
np.random.seed(seed_number)
import random
random.seed(seed_number)

# Previous results data

In [ ]:
# from avellaneda stoikov
best_avellaneda_param_dict={'risk_aversion': 0.01,
 'position_multiplier': 100.0,
 'window_tick': 20.936773236619324,
 'minutes_change_k': 10.0,
 'quantity': 0.0001,
 'k_default': 1.68688291488258,
 'spread_multiplier': 2.0,
 'first_hour': 7.0,
 'last_hour': 19.0}

#from feature importance
most_significant_state_columns=[
 'ask_price_0',
 'ask_price_2',
 'ask_price_4',
 'ask_price_8',
 'ask_qty_0',
 'ask_qty_1',
 'ask_qty_2',
 'ask_qty_7',
 'bid_price_0',
 'bid_price_2',
 'bid_price_4',
 'bid_price_6',
 'bid_price_8',
 'bid_qty_0',
 'bid_qty_1',
 'bid_qty_7',
 'close_0',
 'low_0',
 'ma',
 'microprice_0',
 'microprice_7',
 'midprice_0',
 'spread_0',
 'spread_2',
 'spread_4',
 'spread_8'
]
# most_significant_state_columns=[]

# Configuration

In [ ]:
instrument_pk='btcusdt_binance'

# start_date_pt=datetime.datetime(year=2020, day=8, month=12,hour=9)
# end_date_pt=datetime.datetime(year=2020, day=8, month=12,hour=13)

# start_date_train=end_date_pt+datetime.timedelta(hours=1)#not here
# end_date_train=start_date_train+datetime.timedelta(hours=2)#not here

# start_date_train=datetime.datetime(year=2020, day=8, month=12,hour=9)
# end_date_train=datetime.datetime(year=2020, day=8, month=12,hour=21)
start_date_train=datetime.datetime(year=2020, day=8, month=12,hour=9)
end_date_train=datetime.datetime(year=2020, day=8, month=12,hour=16)
#complete day of training

start_date_test=datetime.datetime(year=2020, day=9, month=12,hour=9)
end_date_test=datetime.datetime(year=2020, day=9, month=12,hour=11)


start_date_test_1=end_date_test+datetime.timedelta(minutes=10)#not here
end_date_test_1=start_date_test_1+datetime.timedelta(hours=2)#not here

start_date_test_2=end_date_test_1+datetime.timedelta(minutes=10)#not here
end_date_test_2=start_date_test_2+datetime.timedelta(hours=2)#not here

start_date_test_3=end_date_test_2+datetime.timedelta(minutes=10)#not here
end_date_test_3=start_date_test_3+datetime.timedelta(hours=2)#not here

# start_date_test_4=end_date_test_3+datetime.timedelta(minutes=10)#not here
# end_date_test_4=start_date_test_4+datetime.timedelta(hours=2)#not here


In [ ]:
tick = TickDB()
depth_data = tick.get_depth(instrument_pk=instrument_pk,start_date=start_date_train,end_date=end_date_test_3)
trade_data = tick.get_trades(instrument_pk=instrument_pk,start_date=start_date_train,end_date=end_date_test_3)

In [ ]:
if HURST_TEST:
    from hurst import compute_Hc, random_walk
    ###hurst exponents
    # H = 0.5 — Brownian motion,
    # 0.5 < H < 1.0 — persistent behavior,
    # 0 < H < 0.5 — anti-persistent behavior.
    ts=candle_data['price']['close']
    hurst_table = {}

    H_train, c, data = compute_Hc(ts.loc[start_date_train:end_date_train], kind='price', simplified=True)
    H_test1, c, data = compute_Hc(ts.loc[start_date_test:end_date_test], kind='price', simplified=True)
    H_test2, c, data = compute_Hc(ts.loc[start_date_test_1:end_date_test_1], kind='price', simplified=True)
    H_test3, c, data = compute_Hc(ts.loc[start_date_test_2:end_date_test_2], kind='price', simplified=True)
    H_test4, c, data = compute_Hc(ts.loc[start_date_test_3:end_date_test_3], kind='price', simplified=True)
#     H_test5, c, data = compute_Hc(ts.loc[start_date_test_4:end_date_test_4], kind='price', simplified=True)

    hurst_table['train']= [H_train,'trending']
    hurst_table['test1']=[H_test1,'trending']
    hurst_table['test2']=[H_test2,'trending']
    hurst_table['test3']=[H_test3,'trending']
    hurst_table['test4']=[H_test4,'mean_reversion']
#     hurst_table['test5']=[H_test5,'mean_reversion']
    hurst_df = pd.DataFrame.from_dict(hurst_table).T
    hurst_df.columns=['Hurst exponent','classification']
    hurst_df

In [ ]:
if HURST_TEST:
    print(f"train hurst:{H_train}")
    print(f"test1 hurst:{H_test1}")
    print(f"test2 hurst:{H_test2}")
    print(f"test3 hurst:{H_test3}")
    print(f"test4 hurst:{H_test4}")
#     print(f"test5 hurst:{H_test5}")

In [ ]:


import plotly.graph_objects as go
if PLOT_CANDLES:
    candle_data = trade_data.resample('1Min').ohlc()
    candle_data_plot = candle_data.loc[start_date_train:end_date_test].append(candle_data.loc[start_date_test_1:end_date_test_4])
    candle_data_plot['price_open']=candle_data_plot['price']['open']
    candle_data_plot['price_high']=candle_data_plot['price']['high']
    candle_data_plot['price_low']=candle_data_plot['price']['low']
    candle_data_plot['price_close']=candle_data_plot['price']['close']
    


    candlestick = go.Candlestick(
        x=candle_data_plot.index,
        open=candle_data_plot['price_open'],
        high=candle_data_plot['price_high'],
        low=candle_data_plot['price_low'],
        close=candle_data_plot['price_close']
    )
    fig = go.Figure(data=[candlestick])

    # plotly line figure
    # fig.update_layout(
    #     title='Pipeline',
    #     yaxis_title=instrument_pk,
    #     shapes = [dict(
    #         x0=end_date_train, x1=end_date_train, y0=0, y1=1, xref='x', yref='paper',
    #         line_width=2)],
    #     annotations=[dict(
    #         x=end_date_train, y=0.05, xref='x', yref='paper',
    #         showarrow=False, xanchor='left', text='end train')]
    # )

    fig.update_layout(
        title='Pipeline',
        yaxis_title=instrument_pk,
        shapes = [
            dict(
            x0=start_date_train, x1=start_date_train, y0=0, y1=1, xref='x', yref='paper',
            line_width=1),

            dict(
            x0=end_date_train, x1=end_date_train, y0=0, y1=1, xref='x', yref='paper',
            line_width=1),
                  dict(
            x0=start_date_test, x1=start_date_test, y0=0, y1=1, xref='x', yref='paper',
            line_width=1),
                  dict(
            x0=end_date_test, x1=end_date_test, y0=0, y1=1, xref='x', yref='paper',
            line_width=1),

                  dict(
            x0=start_date_test_1, x1=start_date_test_1, y0=0, y1=1, xref='x', yref='paper',
            line_width=1),
                  dict(
            x0=end_date_test_1, x1=end_date_test_1, y0=0, y1=1, xref='x', yref='paper',
            line_width=1),

                  dict(
            x0=start_date_test_2, x1=start_date_test_2, y0=0, y1=1, xref='x', yref='paper',
            line_width=1),
                  dict(
            x0=end_date_test_2, x1=end_date_test_2, y0=0, y1=1, xref='x', yref='paper',
            line_width=1),
                  dict(
            x0=start_date_test_3, x1=start_date_test_3, y0=0, y1=1, xref='x', yref='paper',
            line_width=1),
                  dict(
            x0=end_date_test_3, x1=end_date_test_3, y0=0, y1=1, xref='x', yref='paper',
            line_width=1),
#                   dict(
#             x0=start_date_test_4, x1=start_date_test_4, y0=0, y1=1, xref='x', yref='paper',
#             line_width=1),
#                   dict(
#             x0=end_date_test_4, x1=end_date_test_4, y0=0, y1=1, xref='x', yref='paper',
#             line_width=1)




                 ],
        annotations=[
            dict(
            x=start_date_train, y=0.1, xref='x', yref='paper',
            showarrow=False, xanchor='left', text='start train',height = 16),

            dict(
            x=end_date_train, y=0.1, xref='x', yref='paper',
            showarrow=False, xanchor='left', text='end train'),

            dict(
            x=start_date_test, y=0.05, xref='x', yref='paper',
            showarrow=False, xanchor='left', text='start test1'),

            dict(
            x=end_date_test, y=0.1, xref='x', yref='paper',
            showarrow=False, xanchor='left', text='end test1'),


            dict(
            x=start_date_test_1, y=0.05, xref='x', yref='paper',
            showarrow=False, xanchor='left', text='start test2'),

            dict(
            x=end_date_test_1, y=0.1, xref='x', yref='paper',
            showarrow=False, xanchor='left', text='end test2')   ,

            dict(
            x=start_date_test_2, y=0.05, xref='x', yref='paper',
            showarrow=False, xanchor='left', text='start test3'),

            dict(
            x=end_date_test_2, y=0.1, xref='x', yref='paper',
            showarrow=False, xanchor='left', text='end test3')   ,

                    dict(
            x=start_date_test_3, y=0.05, xref='x', yref='paper',
            showarrow=False, xanchor='left', text='start test4'),

            dict(
            x=end_date_test_3, y=0.1, xref='x', yref='paper',
            showarrow=False, xanchor='left', text='end test4')   ,

#                     dict(
#             x=start_date_test_4, y=0.05, xref='x', yref='paper',
#             showarrow=False, xanchor='left', text='start test5'),

#             dict(
#             x=end_date_test_4, y=0.1, xref='x', yref='paper',
#             showarrow=False, xanchor='left', text='end test5') 

        ]
    )


    fig.show()

In [ ]:
def parameter_tuning_benchmark(benchmark_algorithm,start_date,end_date,best_parameter_tuning_dict=None):
    parameters_min = {"risk_aversion":0.01,'window_tick':5,'k_default':0. ,'spread_multiplier':1.}
    parameters_max = {"risk_aversion":0.9, 'window_tick':25,'k_default':2. ,'spread_multiplier':2.}
    
    
    ga_configuration = GAConfiguration
    if DEBUG:
        ga_configuration.population = 15
        ga_configuration_generations=3
        ga_configuration_simultaneous=15
    else:
        ga_configuration.population = 45
        ga_configuration_generations=10
        ga_configuration_simultaneous=25
        
    if best_parameter_tuning_dict is not None:
        ga_configuration.population = 15
        ga_configuration_generations=3
        ga_configuration_simultaneous=5
        pct_moving=0.2
        min_pct=1-pct_moving
        max_pct=1+pct_moving
        parameters_min = {"risk_aversion":best_parameter_tuning_dict["risk_aversion"]*min_pct,'window_tick':int(best_parameter_tuning_dict["window_tick"]*min_pct),'k_default':best_parameter_tuning_dict["k_default"]*min_pct ,'spread_multiplier':best_parameter_tuning_dict["spread_multiplier"]*min_pct}
        parameters_max = {"risk_aversion":best_parameter_tuning_dict["risk_aversion"]*max_pct,'window_tick':int(best_parameter_tuning_dict["window_tick"]*max_pct),'k_default':best_parameter_tuning_dict["k_default"]*max_pct ,'spread_multiplier':best_parameter_tuning_dict["spread_multiplier"]*max_pct}
        
        
    ga_configuration.decay=1/float(ga_configuration_generations)
    
    best_param_dict, summary_df =benchmark_algorithm.parameter_tuning(
        instrument_pk=instrument_pk,
        start_date=start_date,
        end_date=end_date,
        parameters_min=parameters_min,
        parameters_max=parameters_max,
        generations=ga_configuration_generations,
        ga_configuration=ga_configuration,
        max_simultaneous=ga_configuration_simultaneous
    )
    benchmark_algorithm.set_parameters(best_param_dict)
    return benchmark_algorithm,best_param_dict


In [ ]:
# AVELLANEDA STOIKOV - BENCHMARK
parameters_default_as = {
    # Avellaneda default
    "risk_aversion": (0.9),
    "position_multiplier": (100),
    "window_tick": (10),
    "minutes_change_k": (10),
    "quantity": (0.0001),
    "k_default": (0.00769),
    "spread_multiplier": (5.0),
    "first_hour": (7),
    "last_hour": (19),
}

algorithm_info_as='avellaneda_stoikov'
benchmark = AvellanedaStoikov(algorithm_info=algorithm_info_as,parameters=parameters_default_as)
benchmark.set_parameters(best_avellaneda_param_dict)

if PARAMETER_TUNING_BENCHMARK_INITIAL:
    print('launching parameter tuning ')
    benchmark,best_avellaneda_param_dict=parameter_tuning_benchmark(benchmark,start_date=start_date_train,end_date=end_date_train)
    print(f'finished parameter tuning with score {best_avellaneda_param_dict}')
else:
    print(f'Parameter tuning loaded with params {best_avellaneda_param_dict}')

In [ ]:
#DQN Avellaneda stoikov
parameters_default_dqn = {
    # Q
    "skewPricePctAction": [0.,0.05,-0.05,-0.1,0.1],
    "riskAversionAction": [0.01,.1,0.2,0.9],
    "windowsTickAction": [8],
    
    "minPrivateState": (-1),
    "maxPrivateState": (-1),
    "numberDecimalsPrivateState":(3),
    "horizonTicksPrivateState": (5),
    
    "minMarketState": (-1),
    "maxMarketState": (-1),
    "numberDecimalsMarketState": (7),
    "horizonTicksMarketState": (10),
    
    "horizonMinMsTick": (0),
    
    "minCandleState": (-1),
    "maxCandleState": (-1),
    "numberDecimalsCandleState": (3),
    "horizonCandlesState": (2),
    
    "scoreEnum": ScoreEnum.asymmetric_dampened_pnl,
    "timeHorizonSeconds": (5),
    "epsilon": (0.2),#explore prob
    "discountFactor": 0.5,#momentum nesterov nn
    "learningRate": 0.01,#on nn
    # Avellaneda default
    "risk_aversion": (0.9),
    "position_multiplier": (100),
    "window_tick": (10),
    "minutes_change_k": (10),
    "quantity": (0.0001),
    "k_default": (0.00769),
    "spread_multiplier": (5.0),
    "first_hour": (7),
    "last_hour": (19),
    #DQN
    "maxBatchSize": 100000,
    "trainingPredictIterationPeriod": -1,  # train only at the end,offline
    "trainingTargetIterationPeriod": -1,  # train at the end,offline
    "epoch": 2000,
    "stateColumnsFilter": most_significant_state_columns,
    "l1":0.,
    "l2":0.,
    
}

algorithm_info_dqn='avellaneda_stoikov_dqn'

avellaneda_dqn = AvellanedaDQN(algorithm_info=algorithm_info_dqn,parameters=parameters_default_dqn)
avellaneda_dqn.set_parameters(best_avellaneda_param_dict)#same optimization as benchmark


## TRAIN

In [ ]:
start=time.time()
if TRAIN_DQN:    
    if DEBUG:
        algos_per_iteration = 5
        iterations_train=3
        train_simultaneous=5
    else:
        algos_per_iteration = 5
        iterations_train=50
        train_simultaneous=5


    output_train_list=avellaneda_dqn.train(
            instrument_pk=instrument_pk,
            start_date=start_date_train,
            end_date=end_date_train,
            iterations=iterations_train,
            algos_per_iteration=algos_per_iteration,
            simultaneous_algos=train_simultaneous,
        )
    dill.dump_session(session_name)
else:
    print('using previous trained model')
elapsed=time.time()-start
print('train of %d iterations finished in %.2f minutes'%(iterations_train,elapsed/60))

In [ ]:
if TRAIN_DQN:
    memory_replay_out=avellaneda_dqn.get_memory_replay_df("E:\javif\Coding\Python\market_making_fw\python_lambda\output\memoryReplay_AvellanedaDQN_avellaneda_stoikov_dqn_0.csv")
    memory_replay_out.describe()

In [ ]:
if TRAIN_DQN:
    memory_replay_out.head(25)

In [ ]:
files=[]

In [ ]:
if TRAIN_DQN:
    #first iteration results
    name_output=avellaneda_dqn.NAME+'_'+avellaneda_dqn.algorithm_info+'_1'
    backtest_result_train_init=output_train_list[0][name_output]
    plot_1=avellaneda_dqn.plot_trade_results(raw_trade_pnl_df=backtest_result_train_init,plot_open=True)
    
    filename='dqn_train_initial.png'
    plt.savefig(filename)
    files.append(filename)    
    plt.show()
    
    plot_1a=avellaneda_dqn.plot_params(raw_trade_pnl_df=backtest_result_train_init)    
    filename='dqn_train_params_initial.png'
    plt.savefig(filename)
    files.append(filename)        
    plt.show()
    
    
    legends=[]
    for algorithm_iter in range(algos_per_iteration):
        name_output_iter=avellaneda_dqn.NAME+'_'+avellaneda_dqn.algorithm_info+'_%d'%algorithm_iter
        backtest_result_train_init_iter=output_train_list[0][name_output_iter]
        backtest_result_train_init_iter['historicalTotalPnl'].plot()
        legends.append(avellaneda_dqn.algorithm_info+'_%d'%algorithm_iter)
    plt.title('all initial algos')
    plt.legend(legends)    
    filename='dqn_train_all_initial.png'
    plt.savefig(filename)
    files.append(filename)        
    plt.show()
    


In [ ]:
if TRAIN_DQN:    
    iteration_quarter=round(len(output_train_list)/4)
    backtest_result_train_quarter=output_train_list[iteration_quarter][name_output]
    plot_2=avellaneda_dqn.plot_trade_results(raw_trade_pnl_df=backtest_result_train_quarter,plot_open=True)
    
    filename='dqn_train_quarter.png'
    plt.savefig(filename)
    files.append(filename)
    plt.show()
    
    plot_2a=avellaneda_dqn.plot_params(raw_trade_pnl_df=backtest_result_train_quarter)    
    filename='dqn_train_params_quarter.png'
    plt.savefig(filename)
    files.append(filename)
    plt.show()
    
    legends=[]
    for algorithm_iter in range(algos_per_iteration):
        name_output_iter=avellaneda_dqn.NAME+'_'+avellaneda_dqn.algorithm_info+'_%d'%algorithm_iter
        backtest_result_train_mid_iter=output_train_list[iteration_quarter][name_output_iter]
        backtest_result_train_mid_iter['historicalTotalPnl'].plot()
        legends.append(avellaneda_dqn.algorithm_info+'_%d'%algorithm_iter)
    plt.title('all quarter algos')
    plt.legend(legends)    
    filename='dqn_train_all_quarter.png'
    plt.savefig(filename)
    files.append(filename)        
    plt.show()

In [ ]:
if TRAIN_DQN:    
    iteration_middle=round(len(output_train_list)/2)
    backtest_result_train_mid=output_train_list[iteration_middle][name_output]
    plot_2=avellaneda_dqn.plot_trade_results(raw_trade_pnl_df=backtest_result_train_mid,plot_open=True)
    
    filename='dqn_train_middle.png'
    plt.savefig(filename)
    files.append(filename)
    plt.show()
    
    plot_2a=avellaneda_dqn.plot_params(raw_trade_pnl_df=backtest_result_train_mid)    
    filename='dqn_train_params_middle.png'
    plt.savefig(filename)
    files.append(filename)
    plt.show()
    
    legends=[]
    for algorithm_iter in range(algos_per_iteration):
        name_output_iter=avellaneda_dqn.NAME+'_'+avellaneda_dqn.algorithm_info+'_%d'%algorithm_iter
        backtest_result_train_mid_iter=output_train_list[iteration_middle][name_output_iter]
        backtest_result_train_mid_iter['historicalTotalPnl'].plot()
        legends.append(avellaneda_dqn.algorithm_info+'_%d'%algorithm_iter)
    plt.title('all middle algos')
    plt.legend(legends)    
    filename='dqn_train_all_middle.png'
    plt.savefig(filename)
    files.append(filename)        
    plt.show()


In [ ]:
if TRAIN_DQN:    
    iteration_75=round(len(output_train_list)*3/4)
    backtest_result_train_iteration_75=output_train_list[iteration_75][name_output]
    plot_2=avellaneda_dqn.plot_trade_results(raw_trade_pnl_df=backtest_result_train_iteration_75,plot_open=True)
    
    filename='dqn_train_3quarter.png'
    plt.savefig(filename)
    files.append(filename)
    plt.show()
    
    plot_2a=avellaneda_dqn.plot_params(raw_trade_pnl_df=backtest_result_train_iteration_75)    
    filename='dqn_train_params_3quarter.png'
    plt.savefig(filename)
    files.append(filename)
    plt.show()
    
    legends=[]
    for algorithm_iter in range(algos_per_iteration):
        name_output_iter=avellaneda_dqn.NAME+'_'+avellaneda_dqn.algorithm_info+'_%d'%algorithm_iter
        backtest_result_train_mid_iter=output_train_list[iteration_75][name_output_iter]
        backtest_result_train_mid_iter['historicalTotalPnl'].plot()
        legends.append(avellaneda_dqn.algorithm_info+'_%d'%algorithm_iter)
    plt.title('all 3xquarter algos')
    plt.legend(legends)    
    filename='dqn_train_all_3quarter.png'
    plt.savefig(filename)
    files.append(filename)        
    plt.show()

In [ ]:
if TRAIN_DQN:
    #last iteration results
    backtest_result_train_final=output_train_list[-1][name_output]
    plot_3=avellaneda_dqn.plot_trade_results(raw_trade_pnl_df=backtest_result_train_final,plot_open=True)
    filename='dqn_train_final.png'
    plt.savefig(filename)
    files.append(filename)
    plt.show()
    
    plot_3a=avellaneda_dqn.plot_params(raw_trade_pnl_df=backtest_result_train_final)    
    filename='dqn_train_params_final.png'
    plt.savefig(filename)
    files.append(filename)
    plt.show()
    
    legends=[]
    for algorithm_iter in range(algos_per_iteration):
        name_output_iter=avellaneda_dqn.NAME+'_'+avellaneda_dqn.algorithm_info+'_%d'%algorithm_iter
        backtest_result_train_final_iter=output_train_list[-1][name_output_iter]
        backtest_result_train_final_iter['historicalTotalPnl'].plot()
        legends.append(avellaneda_dqn.algorithm_info+'_%d'%algorithm_iter)
    plt.title('all final algos')
    plt.legend(legends)    
    filename='dqn_train_all_final.png'
    plt.savefig(filename)
    files.append(filename)        
    plt.show()

In [ ]:
print('iteration_quarter=%d   iteration_middle=%d   iteration_75=%d'%(iteration_quarter,iteration_middle,iteration_75))

# TEST

In [ ]:
# parameter_test = {}
# parameter_test['epsilon']=0.0
# avellaneda_dqn.set_parameters(parameter_test)
# avellaneda_dqn.parameters

In [ ]:
explore_prob=0.
trainingPredictIterationPeriod=50
trainingTargetIterationPeriod=200
counter_plt_results=0
wining_counter=0
final_results_columns=['instrument','benchmark_open_pnl','benchmark_close_pnl','benchmark_sharpe','benchmark_trades','rl_open_pnl','rl_close_pnl','rl_sharpe','rl_trades']
final_results=pd.DataFrame(columns=final_results_columns)

In [ ]:
def launch_test(start_date_test,end_date_test,is_parameter_tuning_benchmark:bool=False,start_date_train=None,end_date_train=None,best_parameter_tuning_dict:dict=None):
    best_avellaneda_param_dict=None
    if is_parameter_tuning_benchmark:        
        best_benchmark_algorithm,best_avellaneda_param_dict=parameter_tuning_benchmark(benchmark,start_date=start_date_train,end_date=end_date_train,best_parameter_tuning_dict=best_parameter_tuning_dict)
        #set new bet parameter tuning
        benchmark.set_parameters(best_avellaneda_param_dict)
        avellaneda_dqn.set_parameters(best_avellaneda_param_dict)#to be fair optimization is going to optimize avellaneda
        
    
    benchmark_test = benchmark.test(
        instrument_pk=instrument_pk,
        start_date=start_date_test,
        end_date=end_date_test,
    )
    
    dqn_test = avellaneda_dqn.test(
        instrument_pk=instrument_pk,
        start_date=start_date_test,
        end_date=end_date_test,
        explore_prob=explore_prob,
        trainingPredictIterationPeriod=trainingPredictIterationPeriod,
        trainingTargetIterationPeriod=trainingTargetIterationPeriod,
    )
    return benchmark_test,dqn_test,best_avellaneda_param_dict



def plot_results_algorithms(benchmark_test,dqn_test,save_file=True):
    global counter_plt_results
    global wining_counter
    
    name_output=benchmark.algorithm_info
    plot_open=True
    #AS
    backtest_result_test=benchmark_test[name_output]
    plot_bnch=benchmark.plot_trade_results(raw_trade_pnl_df=backtest_result_test,plot_open=plot_open)
    filename='benchmark_test_%d.png'%counter_plt_results
    if save_file:
        plt.savefig(filename)
        files.append(filename)
    plt.show()    
    
    #DQN
    name_output=avellaneda_dqn.algorithm_info
    backtest_result_test=dqn_test[name_output]    
    plot_dqn=avellaneda_dqn.plot_trade_results(raw_trade_pnl_df=backtest_result_test,plot_open=plot_open)
    filename='dqn_test_%d.png'%counter_plt_results
    if save_file:
        plt.savefig(filename)
        files.append(filename)

    plot_dqnA=avellaneda_dqn.plot_params(raw_trade_pnl_df=backtest_result_test)    
    filename='dqn_test_%d_params.png'%counter_plt_results
    if save_file:
        plt.savefig(filename)
        files.append(filename)
    plt.show()
    
    #compare
#     column_to_compare='historicalRealizedPnl'
    column_to_compare='historicalTotalPnl'#
    import copy
    benchmark_result = copy.copy(benchmark_test[benchmark.algorithm_info])
    benchmark_result.set_index('date',inplace=True)
    
    dqn_result = copy.copy(dqn_test[avellaneda_dqn.algorithm_info])
    dqn_result.set_index('date',inplace=True)
    
    
    total_pnl=pd.concat([benchmark_result[column_to_compare],dqn_result[column_to_compare]],axis=1)
    total_pnl.columns=['%s_benchmark'%column_to_compare,'%s_dqn'%column_to_compare]
    total_pnl.sort_index(inplace=True)
    
    total_pnl.fillna(method='ffill',axis=0,inplace=True)
    total_pnl.dropna(inplace=True)
    
    total_pnl.plot(figsize=(17,7))

    sharpe_as=total_pnl['%s_benchmark'%column_to_compare].diff().mean()/total_pnl['%s_benchmark'%column_to_compare].diff().std()
    sharpe_dqn=total_pnl['%s_dqn'%column_to_compare].diff().mean()/total_pnl['%s_dqn'%column_to_compare].diff().std()
    legend=['benchmark sharpe=%.5f  pnl=%.3f'%(sharpe_as,total_pnl['%s_benchmark'%column_to_compare].iloc[-1]),'dqn sharpe=%.5f  pnl=%.3f'%(sharpe_dqn,total_pnl['%s_dqn'%column_to_compare].iloc[-1])]
    if sharpe_dqn>sharpe_as:
        wining_counter+=1
        
    as_open_pnl=benchmark_test[benchmark.algorithm_info]['historicalTotalPnl'].iloc[-1]
    as_close_pnl=benchmark_test[benchmark.algorithm_info]['historicalRealizedPnl'].iloc[-1]
    as_sharpe=sharpe_as
    as_trades=len(benchmark_test[benchmark.algorithm_info])
    
    rl_open_pnl=dqn_test[avellaneda_dqn.algorithm_info]['historicalTotalPnl'].iloc[-1]
    rl_close_pnl=dqn_test[avellaneda_dqn.algorithm_info]['historicalRealizedPnl'].iloc[-1]
    rl_sharpe=sharpe_dqn
    rl_trades=len(dqn_test[avellaneda_dqn.algorithm_info])
    
    
    row=[instrument_pk,as_open_pnl,as_close_pnl,as_sharpe,as_trades,
     rl_open_pnl,rl_close_pnl,rl_sharpe,rl_trades]
    final_results.loc[len(final_results)]=row
        
    plt.title('%s test_%d'%(column_to_compare,counter_plt_results))
    plt.legend(legend)
    plt.xlabel('trade number')
    filename='compare_total_pnl_%d.png'%counter_plt_results
    if save_file:
        plt.savefig(filename)
        files.append(filename)
    plt.show()
    
    
    file_output='dqn_vs_as_output_%d.csv'%counter_plt_results
    files.append(file_output)
    if save_file:
        total_pnl.to_csv(file_output)
    counter_plt_results+=1

### Execute test

In [ ]:

days_test = [
    datetime.datetime(2020,12,9),
    datetime.datetime(2020,12,10),
    datetime.datetime(2020,12,11),
    datetime.datetime(2020,12,12),    
    datetime.datetime(2020,12,13),
    
    datetime.datetime(2020,12,14),
    datetime.datetime(2020,12,15),
    datetime.datetime(2020,12,16),    
    datetime.datetime(2020,12,17),
    datetime.datetime(2020,12,18),
    
    datetime.datetime(2020,12,19),
    datetime.datetime(2020,12,20),
    datetime.datetime(2020,12,21),
    datetime.datetime(2020,12,22),
    datetime.datetime(2020,12,23),
    
    datetime.datetime(2020,12,24),
    datetime.datetime(2020,12,25),
    datetime.datetime(2020,12,26),
    datetime.datetime(2020,12,27),
    datetime.datetime(2020,12,28),
    
    datetime.datetime(2020,12,29),
    datetime.datetime(2020,12,30),
    datetime.datetime(2021,1,1),
    datetime.datetime(2021,1,2),
    datetime.datetime(2021,1,3),
    
    datetime.datetime(2021,1,4),
    datetime.datetime(2021,1,5),
    datetime.datetime(2021,1,6),
    datetime.datetime(2021,1,7),
    datetime.datetime(2021,1,8),
    
]
days_test

In [ ]:
benchmark

In [ ]:

parameter_tuning_every_n_days=5

final_results_columns=['instrument','benchmark_open_pnl','benchmark_close_pnl','benchmark_sharpe','benchmark_trades','rl_open_pnl','rl_close_pnl','rl_sharpe','rl_trades']
final_results=pd.DataFrame(columns=final_results_columns)
start_date_train = None
end_date_train = None
day_counter=0
for day in tqdm.tqdm(days_test):
    start_date = day
    end_date=day
    if start_date_train is None or day_counter%parameter_tuning_every_n_days!=0:
        benchmark_test_temp,dqn_test_temp,best_parameter_dict=launch_test(start_date,end_date) 
        best_parameter_dict=best_avellaneda_param_dict#initially 
    else:
        print(f'Parameter tuning benchmark on day {day_counter} with data from {start_date_train} to {end_date_train}')
        benchmark_test_temp,dqn_test_temp,best_parameter_dict=launch_test(start_date,end_date,is_parameter_tuning_benchmark=PARAMETER_TUNING_BENCHMARK,start_date_train=start_date_train,end_date_train=end_date_train,best_parameter_tuning_dict=best_parameter_dict)
        
        
    plot_results_algorithms(benchmark_test_temp,dqn_test_temp,save_file=False)
    start_date_train=start_date
    end_date_train=end_date
    day_counter+=1
    
final_results['date']=days_test
final_results.set_index('date',inplace=True)    

In [ ]:
final_results['is_better_sharpe']=final_results['rl_sharpe']>final_results['benchmark_sharpe']
final_results['is_better_open_pnl']=final_results['rl_open_pnl']>final_results['benchmark_open_pnl']
final_results['is_better_close_pnl']=final_results['rl_close_pnl']>final_results['benchmark_close_pnl']
final_results

In [ ]:
final_results[['is_better_sharpe','is_better_open_pnl','is_better_close_pnl']].describe()

In [ ]:
final_results[['is_better_sharpe','is_better_open_pnl','is_better_close_pnl']].describe().to_csv('statistical_significance_test_results.csv')
files.append('statistical_significance_test_results.csv')

In [ ]:
final_results.to_csv('statistical_significance_test.csv')
files.append('statistical_significance_test.csv')

In [ ]:
print('win %d times of total %d '%(wining_counter,counter_plt_results))    

In [ ]:
files=list(set(files))
files

In [ ]:
# files=['statistical_significance_test_results.csv','statistical_significance_test.csv']

In [ ]:
wining_counter

In [ ]:
result_str='LOSE'
if (wining_counter>counter_plt_results/2):
    result_str='WIN'
result_str

In [ ]:
save_notebook_session(session_name)
send_email(recipient='javifalces@gmail.com',subject='[%s]PhdScript finished %s -> win ratio %d/%d '%(datetime.datetime.today(),result_str,wining_counter,counter_plt_results),html=final_results_modified.to_html(),body='',file_append=files)